In [ ]:
import math
from dataclasses import dataclass
from typing import Tuple, Sequence
import numpy as np
import pandas as pd

In [2]:
@dataclass
class Params:
    a: float                 # drift of Y_t between jumps
    sigma: float             # diffusion vol of Y_t
    hat_lambda: Tuple[float, float]  # (λ1, λ2) constant jump intensities
    delta: Tuple[float, float]       # (δ1, δ2) shift (sizes are -(δ+Exp(η)))
    eta: Tuple[float, float]         # (η1, η2) exponential rates


In [3]:
def psi(kappa: float, p: Params) -> float:
    a, s = p.a, p.sigma
    (l1, l2) = p.hat_lambda
    (d1, d2) = p.delta
    (e1, e2) = p.eta
    val = 0.5*s*s*kappa*kappa - a*kappa
    val += l1 * ( math.exp(-kappa*d1) * (e1/(e1 + kappa)) - 1.0 )
    val += l2 * ( math.exp(-kappa*d2) * (e2/(e2 + kappa)) - 1.0 )
    return val


In [4]:
def psi_prime_at_zero(p: Params) -> float:
    (l1, l2) = p.hat_lambda
    (d1, d2) = p.delta
    (e1, e2) = p.eta
    return -p.a - l1*(d1 + 1.0/e1) - l2*(d2 + 1.0/e2)


In [5]:
def compute_R(p: Params, tol: float = 1e-12, maxit: int = 200) -> float:
    if psi_prime_at_zero(p) >= 0:
        return 0.0
    lo = 1e-12
    if psi(lo, p) >= 0: lo = 1e-6
    hi = 1.0
    it = 0
    while psi(hi, p) < 0 and it < maxit:
        hi *= 2.0
        it += 1
    # bisection
    for _ in range(maxit):
        mid = 0.5*(lo + hi)
        f = psi(mid, p)
        if abs(f) < tol or (hi - lo) < tol:
            return mid
        if f < 0: lo = mid
        else:     hi = mid
    return 0.5*(lo + hi)


In [6]:
def Phi_of_q(q: float, p: Params, R: float = None, tol: float = 1e-12, maxit: int = 200) -> float:
    if q < 0: raise ValueError("q must be >= 0")
    if R is None: R = compute_R(p)
    if q == 0: return R
    lo = max(R, 0.0)
    hi = max(1.0, 10.0 + lo)
    it = 0
    while psi(hi, p) < q and it < maxit:
        hi *= 2.0
        it += 1
    if psi(hi, p) < q:
        raise RuntimeError("Failed to bracket Phi(q).")
    for _ in range(maxit):
        mid = 0.5*(lo + hi)
        f = psi(mid, p) - q
        if abs(f) < tol or (hi - lo) < tol:
            return mid
        if f < 0: lo = mid
        else:     hi = mid
    return 0.5*(lo + hi)


In [7]:
def stehfest_coeffs(N: int):
    if N % 2 or N <= 0:
        raise ValueError("N must be a positive even integer")
    V = [0.0]*(N+1)
    M = N//2
    fact = math.factorial
    for k in range(1, N+1):
        s = 0.0
        j_min = (k+1)//2
        j_max = min(k, M)
        for j in range(j_min, j_max+1):
            num = (j**M) * fact(2*j)
            den = fact(M - j) * fact(j) * fact(j - 1) * fact(k - j) * fact(2*j - k)
            s += num / den
        V[k] = s * ((-1)**(k + M))
    return V


In [8]:
def cdf_first_hit(T: float, y: float, p: Params, R: float = None, N: int = 12) -> float:
    """
    Returns F(T,y) = P(τ <= T).
    Uses Gaver–Stehfest with order N (even). Monotone in T (↑) and y (↓).
    """
    if T <= 0: return 0.0
    if R is None: R = compute_R(p)
    V = stehfest_coeffs(N)
    ln2_over_T = math.log(2.0) / T

    ssum = 0.0
    for k in range(1, N+1):
        qk = k * ln2_over_T
        Phi = Phi_of_q(qk, p, R=R)
        Lhat = math.exp(-y * Phi) / qk
        ssum += V[k] * Lhat

    val = ln2_over_T * ssum
    # numerical safety clamp
    return 0.0 if val < 0 else (1.0 if val > 1.0 else val)


In [21]:
def survival_prob(T: float, y: float, p: Params, R: float = None, N: int = 12) -> float:
    return 1.0 - cdf_first_hit(T, y, p, R=R, N=N)

def cdf_grid(T_vals: Sequence[float], y_vals: Sequence[float], p: Params, N: int = 12):
    R = compute_R(p)
    T_vals = np.asarray(list(T_vals), float)
    y_vals = np.asarray(list(y_vals), float)
    C = np.zeros((len(y_vals), len(T_vals)), float)
    for i, y in enumerate(y_vals):
        for j, T in enumerate(T_vals):
            C[i, j] = cdf_first_hit(T, float(y), p, R=R, N=N)
    return C, T_vals, y_vals, R


In [107]:
p = Params(
    a=0.05, sigma=0.8,
    hat_lambda=(100.0, 100.0),
    delta=(0.2, 0.3),
    eta=(2.0, 2.0)
)

T_vals = np.linspace(0.00, 20, 41)
LTV_arr = np.linspace(0.85, .95, 11)

y_vals = np.log(1.0 / LTV_arr )
C, T_vals, Y_vals, R = cdf_grid(T_vals, y_vals, p, N=14)

In [108]:
CDF = pd.DataFrame(C, index=LTV_arr, columns=T_vals)

In [109]:
CDF.columns.name = "T"
CDF.index.name = "LTV"

In [ ]:
import plotly.graph_objects as go

In [111]:
fig = go.Figure(data=[go.Surface(z=CDF, x=T_vals, y=LTV_arr)])
fig.update_layout(
    title="First-Hitting Time CDF  F(T, y) = P(τ ≤ T)  (rotatable)",
    scene=dict(
        xaxis_title="Horizon T",
        yaxis_title="Initial LTV",
        zaxis_title="P(τ ≤ T)"
    ),
    width=900,
    height=650
)

# Show in notebook and also save to an HTML file
fig.show()